# 1. Data Collection
The starting point is two lists of Steam64 IDs extracted from: https://tracker.gg/csgo/ & https://vaclist.net/banned

In [24]:
import pandas as pd
import requests

# Lists with Account IDs
banned = pd.read_excel('banned.xlsx', header=None, squeeze=True).tolist()
experienced = pd.read_excel('experienced.xlsx', header=None, squeeze=True).tolist()

print("Banned Accounts:",len(banned))
print("Experienced Accounts:",len(experienced))

Banned Accounts: 1274
Experienced Accounts: 1397


In [26]:
# Set API parameters & Check VAC Bans to filter data
api_key = 'X' # Replace with your own API key if you are going to use this code
app_id = "730" # Counter-Strike: Global Offensive

vacbanned = []
for steam_id in banned:
    url = f'https://api.steampowered.com/ISteamUser/GetPlayerBans/v1/?key={api_key}&steamids={steam_id}'
    response = requests.get(url)
    data = response.json()
    try:
        if data["players"][0]["VACBanned"]:
            vacbanned.append(steam_id)
    except (KeyError, IndexError):
        continue       

In [29]:
print("Accounts VAC-Banned:",len(vacbanned))

Accounts VAC-Banned: 484


In [28]:
# Create dataframe with banned player's stats
banned_stats = []
for steam_id in vacbanned:
    url = f'https://api.steampowered.com/ISteamUserStats/GetUserStatsForGame/v2/?appid={app_id}&key={api_key}&steamid={steam_id}'
    response = requests.get(url)
    data = response.json()
    if 'playerstats' in data:
        playerstats = data['playerstats']
        if 'stats' in playerstats:
            stats_data = playerstats['stats']
            stats_dict = {'steam_id': steam_id}
            for stat in stats_data:
                stats_dict[stat['name']] = stat['value']
            banned_stats.append(stats_dict)
            
#Converting to df and adding y = 1
banned_df = pd.DataFrame(banned_stats)
banned_df['Banned'] = 1

In [31]:
x,y = banned_df.shape
print("VAC-Banned CS:GO Accounts:",x,"\nFeatures:",y)

VAC-Banned CS:GO Accounts: 263 
Features: 243


In [42]:
# Create dataframe with experienced player's stats
experienced_stats = []
for steam_id in experienced[1074:]:
    url = f'https://api.steampowered.com/ISteamUserStats/GetUserStatsForGame/v2/?appid={app_id}&key={api_key}&steamid={steam_id}'
    response = requests.get(url)
    data = response.json()
    if 'playerstats' in data:
        playerstats = data['playerstats']
        if 'stats' in playerstats:
            stats_data = playerstats['stats']
            stats_dict = {'steam_id': steam_id}
            for stat in stats_data:
                stats_dict[stat['name']] = stat['value']
            experienced_stats.append(stats_dict)
            
# Converting to df and adding y = 0
experienced_df = pd.DataFrame(experienced_stats)
experienced_df['Banned'] = 0

In [44]:
x,y = experienced_df.shape
print("Experienced CS:GO Accounts:",x,"\nFeatures:",y)

Experienced CS:GO Accounts: 1358 
Features: 268


In [51]:
# Create dataset to train Machine Learning model
merged_df = pd.concat([banned_df, experienced_df], axis=0)
merged_df = merged_df.reset_index(drop=True)
merged_df["steam_id"] = merged_df["steam_id"].astype(str)
merged_df.to_excel("dataset.xlsx",index=False)